In [1]:
for ii in 1:4
    addprocs(["heathred"])
end

for ii in 1:1
    addprocs(["amon"], dir="")
    addprocs(["zeus"], dir="")
    addprocs(["jove"], dir="")
    addprocs(["ares"], dir="")
end

for ii in 1:10
    #addprocs(["uggp"], dir="")
end

addprocs(2)
workers()

expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error


10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [2]:
using Iterators
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [3]:
@everywhere using RecursiveAutoencoders
@everywhere  using UnfoldingRAE

using ClusterSoup

In [4]:
using WordEmbeddings
LL,word_indexes, indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(50,268810)
Dict{String,Int64}
Array{String,1}


In [5]:
training_trees = open("training_sents.jsz","r") do fs
    deserialize(fs)
end;
@assert length(@pipe training_trees |> filter(x->typeof(x)===UTF8String,_))==0
@pz training_trees

training_trees		Array{Any,1}	(3913,)


In [6]:
r_training_trees = r_chunk_data(training_trees)

10-element Array{RemoteRef,1}:
 RemoteRef(2,1,172) 
 RemoteRef(3,1,174) 
 RemoteRef(4,1,176) 
 RemoteRef(5,1,178) 
 RemoteRef(6,1,180) 
 RemoteRef(7,1,182) 
 RemoteRef(8,1,184) 
 RemoteRef(9,1,186) 
 RemoteRef(10,1,188)
 RemoteRef(11,1,190)

In [7]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e = reshape(θ[1: W_e_len],W_e_shape)
    rae.b_e = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d = reshape(θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len],W_d_shape)
    rae.b_d = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    pack(rae.W_e,rae.b_e, rae.W_d,rae.b_d)
end

@everywhere function pack(∇W_e::Matrix{Float64}, ∇b_e::Vector{Float64}, ∇W_d::Matrix{Float64}, ∇b_d::Vector{Float64})
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end


In [8]:

function loss!(θ::Vector)  
    #warn("loss! not defined")
    grad = similar(θ)
    #loss_and_loss_grad!(θ::Vector)
    cached_loss_and_loss_grad!(θ, grad)
end

function loss_grad!(θ::Vector, storage::Vector) 
    #warn("loss_grad not defined")
    cached_loss_and_loss_grad!(θ, grad)
end


rae_outer = RAE(LL,word_indexes,indexed_words);
r_rae_outers = put!(workers(), rae_outer)
function loss_and_loss_grad!(θ::Vector, grad::Vector)   
    grad[:] = 0
    @inbounds for r_rae_outer in r_rae_outers
        update_remote(r_rae_outer, rae->unpack!(rae, θ) )
    end
    
    function get_remote_loss_grad_function(r_rae_outer::RemoteRef)
        @assert r_rae_outer.where == myid()
        rae = fetch(r_rae_outer)        
        function loss_and_loss_grad(tree::(Any,Any))
            Δs, err = UnfoldingRAE.loss_and_loss_grad(rae, tree)
            [pack(Δs...), err]
        end
    end
    loss_and_loss_grads = map(r_rae_outers) do r_raeouter
        remotecall(r_raeouter.where, get_remote_loss_grad_function, r_raeouter)
    end
    
    
    ret = prechunked_mapreduce(r_training_trees, loss_and_loss_grads, (+)) 
    grad[:] = ret[1:end-1]
    err=ret[end]
    
    grad[:]/=length(training_trees)
    err/=length(training_trees)
    err
end

loss_and_loss_grad! (generic function with 1 method)

In [9]:

loss_and_loss_grad_cache = Dict{Vector{Float64},(Float64, Vector{Float64})}()
loss_and_loss_grad_cache_hits = 0
loss_and_loss_grad_cache_misses = 0
function cached_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global loss_and_loss_grad_cache
    global loss_and_loss_grad_cache_hits
    global loss_and_loss_grad_cache_misses
    if haskey(loss_and_loss_grad_cache,θ)
        loss_and_loss_grad_cache_hits+=1
        err, grad[:] = loss_and_loss_grad_cache[θ]
        err
    else
        loss_and_loss_grad_cache_misses+=1
        err = loss_and_loss_grad!(θ, grad)
        loss_and_loss_grad_cache[θ] = (err, grad)
        err
    end
end

cached_loss_and_loss_grad! (generic function with 1 method)

In [10]:
using NLopt

f_call_count = 0
function tracking_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global f_call_count
    f_call_count+=1
    f_val = cached_loss_and_loss_grad!(θ, grad)   
    println(f_call_count, '\t',f_val,'\t',norm(grad))
    f_val
end
#:LD_MMA, :LD_CCSAQ, :LD_LBFGS, :LD_SLSQP, :LD_VAR2, :LD_VAR1, :LD_TNEWTON_RESTART
opt = Opt(:LD_MMA, length(pack(rae_outer)))

#ftol_abs!(opt,1e-9)
maxtime!(opt, 30)
min_objective!(opt, tracking_loss_and_loss_grad!)
(optf,optx,ret) = optimize!(opt,pack(rae_outer))


exception on 6: exception on exception on 8: 9: exception on 4: ERROR: NLopt not properly installed. Please run Pkg.build("NLopt")
 in error at error.jl:21
 in include_string at loading.jl:97
 in include_from_node1 at ./loading.jl:131
 in reload_path at loading.jl:152
 in _require at loading.jl:67
 in anonymous at multi.jl:1284
 in anonymous at multi.jl:848
 in run_work_thunk at multi.jl:621
 in run_work_thunk at multi.jl:630
 in anonymous at task.jl:6
while loading /home/wheel/oxinabox/.julia/v0.3/NLopt/src/NLopt.jl, in expression starting on line 16
ERROR: NLopt not properly installed. Please run Pkg.build("NLopt")
 in error at error.jl:21
 in include_string at loading.jl:97
 in include_from_node1 at ./loading.jl:131
 in reload_path at loading.jl:152
 in _require at loading.jl:67
 in anonymous at multi.jl:1284
 in anonymous at multi.jl:848
 in run_work_thunk at multi.jl:621
 in run_work_thunk at multi.jl:630
 in anonymous at task.jl:6
while loading /home/wheel/oxinabox/.julia/v0.3/NL

1	631.6659266366264	6.040049454477725e12
2	679.6542505423596	3.3760902485548275e14
3	679.7047992214096	3.294383070608061e14


(631.6659266366264,[-0.00822333,-0.0112657,0.0139677,0.00610842,-0.00242872,0.00818049,-0.00365157,-0.0162207,-0.011425,0.00250472  …  -0.00452749,-0.00962091,-0.00299901,-0.00132408,0.00980096,-0.00155211,-0.00441812,-0.0022725,0.0137535,-0.0105671],:MAXTIME_REACHED)

In [11]:
optf

631.6659266366264

In [12]:
#MMA
#167	623.6986598947503	3.472825423124687e12

In [17]:
using Optim #https://github.com/JuliaOpt/Optim.jl
f=DifferentiableFunction(loss!,loss_grad!,cached_loss_and_loss_grad!)

res = optimize(f, pack(rae, method=:l_bfgs, show_trace = true, store_trace = true, iterations = 30);
@printval res.f_calls 
@printval res.g_calls 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.x_converged 

@printval res.trace
@printval loss_and_loss_grad_cache_hits
@printval loss_and_loss_grad_cache_misses

LoadError: res not defined
while loading In[17], in expression starting on line 4

In [14]:
res

LoadError: res not defined
while loading In[14], in expression starting on line 1

In [15]:
θ=pack(rae_outer)
@time loss_and_loss_grad!(θ, zeros(θ))

elapsed time: 9.82446358 seconds (3661320 bytes allocated)


631.6659266366263

In [16]:
Profile.clear()